Importing Libariries to work on the project in pyspark

Setting the Directory

In [1]:
import os
import sys
 
os.environ["SPARK_HOME"] = "/usr/hdp/current/spark2-client"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/local/anaconda/bin/python" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/local/anaconda/bin/python"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.4-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

Loading the file to do  Modelling

In [3]:
df = spark.read.csv('winequality-red.csv',header=True)

Dataset Description

The Wine Quality dataset contains information about various physicochemical properties of wines. e are going to download and load the dataset into Python and perform an initial analysis to disclose what is inside it. And applying some Machine learning algorithms.

FEATURES DESCRIPTION

Fixed acidity: It indicates the amount of tartaric acid in wine and is measured in g/dm3

Volatile acidity: It indicates the amount of acetic acid in the wine. It is measured in g/dm3.

Citric acid: It indicates the amount of citric acid in the wine. It is also measured in g/dm3

Residual sugar: It indicates the amount of sugar left in the wine after the fermentation process is done. It is also measured in g/dm3

Free sulfur dioxide: It measures the amount of sulfur dioxide (SO2) in free form. It is also measured in g/dm3

Total sulfur dioxide: It measures the total amount of SO2 in the wine. This chemical works as an antioxidant and antimicrobial agent.

Density: It indicates the density of the wine and is measured in g/dm3.

pH: It indicates the pH value of the wine. The range of value is between 0 to 14.0, which indicates very high acidity, and 14 indicates basic acidity.

Sulphates: It indicates the amount of potassium sulphate in the wine. It is also measured in g/dm3.

Alcohol: It indicates the alcohol content in the wine.

Quality: It indicates the quality of the wine, which is ranged from 1 to 10. Here, the higher the value is, the better the wine.

In [4]:
df.show()

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+-----+---------+-------+-------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|   pH|sulphates|alcohol|quality|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+-----+---------+-------+-------+
|        7.400|           0.700|      0.000|         1.900|    0.076|             11.000|              34.000|  0.998|3.510|    0.560|  9.400|  5.000|
|        7.800|           0.880|      0.000|         2.600|    0.098|             25.000|              67.000|  0.997|3.200|    0.680|  9.800|  5.000|
|        7.800|           0.760|      0.040|         2.300|    0.092|             15.000|              54.000|  0.997|3.260|    0.650|  9.800|  5.000|
|       11.200|           0.280|      0.560|         1.900|    0.075|             17.000|     

Checking for null values

In [5]:
for col in df.columns:
    print("no. of cells in column", col, "with null values:", df.filter(df[col].isNull()).count())

no. of cells in column fixed acidity with null values: 0
no. of cells in column volatile acidity with null values: 0
no. of cells in column citric acid with null values: 0
no. of cells in column residual sugar with null values: 0
no. of cells in column chlorides with null values: 0
no. of cells in column free sulfur dioxide with null values: 0
no. of cells in column total sulfur dioxide with null values: 0
no. of cells in column density with null values: 0
no. of cells in column pH with null values: 0
no. of cells in column sulphates with null values: 0
no. of cells in column alcohol with null values: 0
no. of cells in column quality with null values: 0


No null values in the dataset

In [6]:
df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
fixed acidity,1599,8.319637273295838,1.7410963181276948,10.000,9.900
volatile acidity,1599,0.5278205128205131,0.17905970415353525,0.120,1.580
citric acid,1599,0.2709756097560964,0.19480113740531824,0.000,1.000
residual sugar,1599,2.5388055034396517,1.40992805950728,0.900,9.000
chlorides,1599,0.08746654158849257,0.047065302010090085,0.012,0.611
free sulfur dioxide,1599,15.874921826141339,10.46015696980971,1.000,9.000
total sulfur dioxide,1599,46.46779237023139,32.89532447829907,10.000,99.000
density,1599,0.9967517198248945,0.0018906979941762342,0.990,1.004
pH,1599,3.311113195747343,0.15438646490354271,2.740,4.010


In [7]:
df.printSchema()

root
 |-- fixed acidity: string (nullable = true)
 |-- volatile acidity: string (nullable = true)
 |-- citric acid: string (nullable = true)
 |-- residual sugar: string (nullable = true)
 |-- chlorides: string (nullable = true)
 |-- free sulfur dioxide: string (nullable = true)
 |-- total sulfur dioxide: string (nullable = true)
 |-- density: string (nullable = true)
 |-- pH: string (nullable = true)
 |-- sulphates: string (nullable = true)
 |-- alcohol: string (nullable = true)
 |-- quality: string (nullable = true)



Converting my string feilds to double type

In [8]:
from pyspark.sql.types import IntegerType
from pyspark.sql.types import       DoubleType, StructField,IntegerType, StructType,StringType
newDF=[StructField('fixed acidity',DoubleType(),True),
       StructField('volatile acidity',DoubleType(),True),
       StructField('citric acid',DoubleType(),True),
       StructField('residual sugar',DoubleType(),True),
       StructField('chlorides',DoubleType(),True),
       StructField('free sulfur dioxide',DoubleType(),True),
       StructField('total sulfur dioxide',DoubleType(),True),
       StructField('density',DoubleType(),True),
       StructField('pH',DoubleType(),True),
       StructField('sulphates',DoubleType(),True),
       StructField('alcohol',DoubleType(),True),
       StructField('quality',DoubleType(),True)
      
       ]
finalStruct=StructType(fields=newDF)
df=spark.read.csv('winequality-red.csv',header=True,schema=finalStruct)

In [10]:
df.printSchema()

root
 |-- fixed acidity: double (nullable = true)
 |-- volatile acidity: double (nullable = true)
 |-- citric acid: double (nullable = true)
 |-- residual sugar: double (nullable = true)
 |-- chlorides: double (nullable = true)
 |-- free sulfur dioxide: double (nullable = true)
 |-- total sulfur dioxide: double (nullable = true)
 |-- density: double (nullable = true)
 |-- pH: double (nullable = true)
 |-- sulphates: double (nullable = true)
 |-- alcohol: double (nullable = true)
 |-- quality: double (nullable = true)



Extracting my X and Y variable using Vectoe Assembler

In [11]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=["fixed acidity","volatile acidity","citric acid","residual sugar","chlorides","free sulfur dioxide","total sulfur dioxide","density","pH","sulphates","alcohol"],
                            outputCol="features")
feature_vec=assembler.transform(df)




In [12]:
feature_vec.select("features")

DataFrame[features: vector]

In [13]:
# Split the data into train and test sets
train_data, test_data = feature_vec.randomSplit([.75,.25],seed=0)

In [14]:
from pyspark.ml.classification import LogisticRegression

# Create initial LogisticRegression model
lr = LogisticRegression(labelCol="quality", featuresCol="features",  
                        maxIter=100, regParam=0.0001, family="multinomial",  
                        elasticNetParam=0.0)

# Train model with Training Data
lrModel = lr.fit(train_data)
predictions = lrModel.transform(test_data)
predictions.printSchema()

root
 |-- fixed acidity: double (nullable = true)
 |-- volatile acidity: double (nullable = true)
 |-- citric acid: double (nullable = true)
 |-- residual sugar: double (nullable = true)
 |-- chlorides: double (nullable = true)
 |-- free sulfur dioxide: double (nullable = true)
 |-- total sulfur dioxide: double (nullable = true)
 |-- density: double (nullable = true)
 |-- pH: double (nullable = true)
 |-- sulphates: double (nullable = true)
 |-- alcohol: double (nullable = true)
 |-- quality: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = true)



In [15]:
# Train model with Training Data
lrModel = lr.fit(train_data)


In [16]:
predictions = lrModel.transform(test_data)
predictions.printSchema()

root
 |-- fixed acidity: double (nullable = true)
 |-- volatile acidity: double (nullable = true)
 |-- citric acid: double (nullable = true)
 |-- residual sugar: double (nullable = true)
 |-- chlorides: double (nullable = true)
 |-- free sulfur dioxide: double (nullable = true)
 |-- total sulfur dioxide: double (nullable = true)
 |-- density: double (nullable = true)
 |-- pH: double (nullable = true)
 |-- sulphates: double (nullable = true)
 |-- alcohol: double (nullable = true)
 |-- quality: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = true)



In [17]:
predictions.select('quality','prediction').show()

+-------+----------+
|quality|prediction|
+-------+----------+
|    4.0|       6.0|
|    6.0|       7.0|
|    8.0|       7.0|
|    5.0|       5.0|
|    7.0|       6.0|
|    6.0|       6.0|
|    7.0|       6.0|
|    5.0|       6.0|
|    7.0|       6.0|
|    5.0|       6.0|
|    8.0|       6.0|
|    6.0|       6.0|
|    6.0|       6.0|
|    6.0|       6.0|
|    5.0|       5.0|
|    5.0|       5.0|
|    6.0|       6.0|
|    6.0|       5.0|
|    6.0|       6.0|
|    5.0|       5.0|
+-------+----------+
only showing top 20 rows



In [18]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='quality', metricName='accuracy')
evaluator.evaluate(predictions)

0.5725593667546174

In [19]:
evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='quality', metricName='f1')
evaluator.evaluate(predictions)

0.5365761408221408

In [20]:
predictions.groupBy('quality').count().show()

+-------+-----+
|quality|count|
+-------+-----+
|    8.0|    2|
|    7.0|   55|
|    4.0|   11|
|    3.0|    1|
|    6.0|  160|
|    5.0|  150|
+-------+-----+



From the above Logistics model model we can see that Accuracy score is 53, In the next step we are going to do Grid search to see hyper tunned results for the model

In [21]:
#Grid Search
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

paramGrid = (ParamGridBuilder()\
             .addGrid(lr.regParam,[0.001,0.01,0.1,1])\
             .addGrid(lr.elasticNetParam,[0.0,0.5,1.0])\
             .build())

# Create 4-fold CrossValidator
cv = CrossValidator(estimator=lr, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=4)

cvModel = cv.fit(train_data)

In [22]:
list(zip(cvModel.avgMetrics, cvModel.getEstimatorParamMaps()))

[(0.5908128356912047,
  {Param(parent='LogisticRegression_433fa381ab7c7d2304c9', name='regParam', doc='regularization parameter (>= 0).'): 0.001,
   Param(parent='LogisticRegression_433fa381ab7c7d2304c9', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0}),
 (0.5520092249437069,
  {Param(parent='LogisticRegression_433fa381ab7c7d2304c9', name='regParam', doc='regularization parameter (>= 0).'): 0.001,
   Param(parent='LogisticRegression_433fa381ab7c7d2304c9', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.5}),
 (0.5440181922370475,
  {Param(parent='LogisticRegression_433fa381ab7c7d2304c9', name='regParam', doc='regularization parameter (>= 0).'): 0.001,
   Param(parent='LogisticRegression_433fa381ab7c7d2304c9', name='elasticNetParam', doc='the ElasticNet m

In [23]:
#Best Model Params
score_params_list = list(zip(cvModel.avgMetrics, cvModel.getEstimatorParamMaps()))
max(score_params_list,key=lambda item:item[0])

(0.5908128356912047,
 {Param(parent='LogisticRegression_433fa381ab7c7d2304c9', name='regParam', doc='regularization parameter (>= 0).'): 0.001,
  Param(parent='LogisticRegression_433fa381ab7c7d2304c9', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0})

In [24]:
predictions = cvModel.bestModel.transform(test_data)

In [25]:
evaluator.evaluate(predictions)

0.5492386419193881

From the above Logistics regression  model we can see that Accuracy score using pyspark is 54.92 and accuracy score for same model using Sklearn is 59.75 for the dataset